In [1]:
import os
import torch
from torchvision import datasets
from torchvision import transforms

from utils.datasets import get_knn_evaluation_loader

from utils.networks import SimCLR, DecoderNN_1input, DecoderNN_1input_one_branch, build_resnet18, build_resnet50
from utils.transforms import *

/home/nazim/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
decoder = DecoderNN_1input(
    transforms=list(transformations_dict.keys()),
    num_discrete_magnitude=NUM_DISCREATE,
    device=device,
    use_proba_head=False
).cuda()

In [31]:
from copy import deepcopy

In [120]:
p = 0.25
batch_size = 16


log_p, actions_index, entropy = decoder(batch_size=batch_size)


ids = torch.tensor([ range(batch_size), range(batch_size) ])
new_ids = ids.clone()
new_action_index = deepcopy(actions_index)

if p != 0:
    
    L = int(batch_size*p)
    
    # Get random indices of the transformations that are going to be permuted
    indices = list(range(batch_size))
    random.shuffle(indices)
    indices = sorted(indices[:L])
    
    # print(indices)
    
    # original transformations indices of format: (branch[0,1], n) 
    indices_2d = [(0, i) for i in indices] + [(1, i) for i in indices]
    
    # shuffle  the original transformations indices
    new_indices_2d = indices_2d.copy()
    random.shuffle(new_indices_2d)
    
    # Permute the samples
    for (i, j), (new_i, new_j) in zip(indices_2d, new_indices_2d):
        # print((i, j), (new_i, new_j))
        new_ids[i, j] = ids[new_i, new_j]
        new_action_index[j][i] = new_action_index[new_j][new_i]


print(new_ids)

rewards = torch.arange(start=0, end=batch_size).float()

tensor([[ 0, 13,  2,  3,  4,  5,  6,  7, 13,  9, 10, 11, 12,  7, 14, 15],
        [ 0,  8,  2,  3,  4,  5,  6,  1,  8,  9, 10, 11, 12,  1, 14, 15]])


In [121]:
rewards_x = (rewards[new_ids[0]] + rewards[new_ids[1]]) / 2
rewards_x

tensor([ 0.0000, 10.5000,  2.0000,  3.0000,  4.0000,  5.0000,  6.0000,  4.0000,
        10.5000,  9.0000, 10.0000, 11.0000, 12.0000,  4.0000, 14.0000, 15.0000])

tensor([ 8.0000,  5.5000,  8.0000,  8.0000,  4.0000,  5.0000,  6.0000,  6.5000,
         8.0000,  5.5000, 10.0000,  0.5000, 12.0000,  4.0000, 14.0000, 15.0000])

In [94]:
print(ids)
print()
print(new_ids)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]])

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]])
